In [ ]:
%matplotlib inline
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
import azureml
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

## Create AML Workspace

In [ ]:
from azureml.core import Workspace
ws = Workspace.create(name='unetworkspace',
                      subscription_id='your_sub_id', 
                      resource_group='unetrg',
                      create_resource_group=True,
                      location='eastus2' 
                     )
ws.write_config(path='.', file_name='aml_config.json')

In [ ]:
## get workspace by its id
#from azureml.core import Workspace
#ws = Workspace.get(name='unetworkspace',subscription_id='your_sub_id')

In [ ]:
## get workspace from local config
#ws = Workspace.from_config('aml_config/aml_config.json')
#print('Workspace name: ' + ws.name, 
#      'Azure region: ' + ws.location, 
#      'Subscription id: ' + ws.subscription_id, 
#      'Resource group: ' + ws.resource_group, sep = '\n')

## Create experiment

In [ ]:
from azureml.core import Experiment
script_folder = './aml_script'
exp = Experiment(workspace=ws, name='aml_script')

## Upload data to datastore

In [ ]:
ds = ws.get_default_datastore()
ds.upload(src_dir='../data/train', target_path='unet', overwrite=True, show_progress=True)

## Create compute target

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=2)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

In [ ]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

## Submit training job to gpucluster

In [ ]:
from azureml.train.dnn import TensorFlow
script_folder = './aml_script'
script_params = {
    '--data-folder': ds.path('unet').as_mount(),
    '--batch-size': 2   
}

est = TensorFlow(source_directory=script_folder,
                 script_params=script_params,
                 compute_target=compute_target, 
                 pip_packages=['keras', 'scikit-image'],
                 entry_script='trainUnet.py', 
                 use_gpu=True)

#est.run_config.environment.python.user_managed_dependencies = True # Means customized all Python env

In [ ]:
run = exp.submit(est)

In [ ]:
# Show training log
run.wait_for_completion(show_output=True)

In [ ]:
# Show result file including log and model
run.get_file_names()

## Download and register model

In [ ]:
for f in run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join('./model', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)

In [ ]:
# register model to AML from experiment
model = run.register_model(model_name='unet', model_path='outputs/model/unet_keras.hdf5')

In [ ]:
## register model from local
#from azureml.core.model import Model
#model = Model.register(model_name = 'unet', model_path = 'model/unet_keras.hdf5', workspace = ws)

## Write score.py

In [ ]:
%%writefile score.py
from __future__ import print_function
import numpy as np
import os
import glob
import logging
import json
from azureml.core.model import Model as Modelaml
from azure.storage.blob import BlockBlobService, PublicAccess

import skimage.io as io
import skimage.transform as trans
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
from keras.preprocessing.image import ImageDataGenerator



def init():
    global model

    model_root = Modelaml.get_model_path('unet')
    model = unet()
    model.load_weights(model_root)
    print('model_loaded')

def unet(pretrained_weights = None,input_size = (256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    if(pretrained_weights):
        model.load_weights(pretrained_weights)
    return model


def testGenerator(testpath, num_image = 30,target_size = (256,256),flag_multi_class = False,as_gray = True):
    for i in range(num_image):
        img = io.imread(os.path.join(testpath,"%d.png"%i),as_gray = as_gray)
        img = img / 255
        img = trans.resize(img,target_size)
        img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
        img = np.reshape(img,(1,)+img.shape)
        yield img

def saveResult(npyfile,flag_multi_class = False,num_class = 2):
    block_blob_service = BlockBlobService(connection_string = "AzureStorage ConnectionString") 
    result_url = "Azure Storage Blob Url"
    container_name ='unetresult'   
    block_blob_service.create_container(container_name) 
    block_blob_service.set_container_acl(container_name, public_access=PublicAccess.Container)
    
    Sky = [128,128,128]
    Building = [128,0,0]
    Pole = [192,192,128]
    Road = [128,64,128]
    Pavement = [60,40,222]
    Tree = [128,128,0]
    SignSymbol = [192,128,128]
    Fence = [64,64,128]
    Car = [64,0,128]
    Pedestrian = [64,64,0]
    Bicyclist = [0,128,192]
    Unlabelled = [0,0,0]
    COLOR_DICT = np.array([Sky, Building, Pole, Road, Pavement, Tree, SignSymbol, Fence, Car, Pedestrian, Bicyclist, Unlabelled])
    
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        io.imsave("predict.png",img)
        # Upload the created file
        block_blob_service.create_blob_from_path(container_name, "%d_predict.png"%i, "predict.png")

    return result_url + container_name

def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i,:] = color_dict[i]
    return img_out / 255

def run(raw_data):
    data_url = json.loads(raw_data)['storage_url']
    data_count = json.loads(raw_data)['count']

    # make prediction
    testGene = testGenerator(data_url, num_image = data_count)
    results = model.predict_generator(testGene,data_count,verbose=1)
    result_url = saveResult(results) # result saved container url
    return result_url

## Write env.yml

In [ ]:
from azureml.core.runconfig import CondaDependencies

#cd = CondaDependencies.create()
cd = CondaDependencies()
#cd.add_channel('default')
#cd.add_channel('conda-forge')
cd.add_tensorflow_pip_package()
cd.add_pip_package('keras')
cd.add_pip_package('scikit-image')
cd.add_pip_package('azure')
cd.add_pip_package('azure-storage-blob')
cd.save_to_file(base_directory='.', conda_file_path='myenv.yml')

print(cd.serialize_to_string())

## Deploy model to Azure Container Instance

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               auth_enabled=True, # this flag generates API keys to secure access
                                               memory_gb=1, 
                                               tags={'name':'unet', 'framework': 'Keras'},
                                               description='Keras Unet')

In [ ]:
from azureml.core.image import ContainerImage

imgconfig = ContainerImage.image_configuration(execution_script="score.py", 
                                               runtime="python", 
                                               conda_file="myenv.yml")

In [ ]:
%%time
from azureml.core.webservice import Webservice

service = Webservice.deploy_from_model(workspace=ws,
                                       name='keras-unet',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=imgconfig)

service.wait_for_deployment(show_output=True)

In [ ]:
# print ACI deployment log
print(service.get_logs())

## Test published service

In [ ]:
import json
# storage_url is the url of Azure storage container url where test images are stored. count is amount of test images.
result = service.run(input_data=json.dumps({'storage_url': 'Azure storage container url', 'count': 2}))
print(result) # print test images stored container url

In [ ]:
# plot the result image
import matplotlib.pyplot as plt
import skimage.io as skio
img = skio.imread(result + "/0_predict.png")
plt.imshow(img, cmap = 'gray')